In [ ]:
1. Install and Import Dependencies

!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

!pip install transformers requests beautifulsoup4 pandas numpy

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

2. Instantiate Model

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Encode and Calculate Sentiment

tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

result = model(tokens)

result.logits

int(torch.argmax(result.logits))+1

4. Collect Reviews

r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

reviews

5. Load Reviews into DataFrame and Score

import numpy as np
import pandas as pd

df = pd.DataFrame(np.array(reviews), columns=['review'])

df['review'].iloc[0]

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

sentiment_score(df['review'].iloc[1])

df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

df

df['review'].iloc[3]
